In [11]:
import cv2
import numpy as np
import torch
import time
import tkinter as tk
from tkinter import ttk
import csv
import os
from twilio.rest import Client

# Constants
FRAME_DECIMATION_FACTOR = 15  # Process every nth frame
TARGET_WIDTH = 640  # Lower resolution width
TARGET_HEIGHT = 480  # Lower resolution height
PERSON_PROXIMITY_THRESHOLD = 200  # Threshold distance for considering a person near an object (in pixels)
TIME_THRESHOLD = 15  # Time threshold for considering a person away from an object (in seconds)

# Your Twilio credentials
account_sid = ''
auth_token = ''
twilio_phone_number = ''  # Replace with your Twilio phone number

# Initialize Twilio client
client = Client(account_sid, auth_token)

# Function to send SMS based on priority
def send_sms(priority):
    # Map priority to corresponding team and message
    team_messages = {
        'security_notification': ('Security team', 'Security notification: Unattended luggage detected.'),
        'maintenance_notification': ('Maintenance team', 'Maintenance notification: Fault detected.'),
        'priority_notification': ('Operations team', 'Priority notification: Please investigate the situation.'),
        # Add more mappings as needed
    }

    # Check if priority is in the mapping
    if priority in team_messages:
        team, message_body = team_messages[priority]

        # Send SMS
        message = client.messages.create(
            body=message_body,
            from_=twilio_phone_number,
            to=''  # Replace with actual recipient number
        )

        print(f"SMS sent to {team}:\n{message.sid}")
    else:
        print("Invalid priority.")

# Function to categorize individuals based on the detected objects and person count
def categorize_traveler(person_count, objects_detected):
    if person_count > 1:
        return "Family or Friends"
    elif person_count == 1 and any(object in objects_detected for object in luggage):
        return "Business Traveler"
    else:
        return "Solo Traveler"

# Function to process frames
def process_frames(cap, model, selected_class, log_file):
    frame_count = 0
    object_history = {}  # Dictionary to store object history

    # Load COCO class labels from file
    with open('coco.names', 'r') as f:
        classes = [line.strip() for line in f.readlines()]

    # Dictionary to store the last time the selected class was detected near a person
    last_near_person_time = {}

    # Dictionary to store the last time the person was detected near an object
    last_near_object_time = {}

    # Initialize notification variables
    security_notification = False
    maintenance_notification = False
    priority_notification = False

    # Check if the log file already exists
    if os.path.exists(log_file):
        os.remove(log_file)

    # Open CSV file for logging
    with open(log_file, mode='w', newline='') as log_file:
        log_writer = csv.writer(log_file)
        log_writer.writerow(['Timestamp', 'Frame', 'People_Count', 'Object_Count', 'Unattended_Objects_Count', 'Business_Traveler', 'Family_or_Friends', 'Solo_Traveler', 'Category', 'Class', 'Location', 'Security_Notification', 'Maintenance_Notification', 'Priority_Notification'])

        while True:
            ret, frame = cap.read()
            if not ret:
                break
            frame_count += 1
            if frame_count % FRAME_DECIMATION_FACTOR != 0:
                continue

            # Reset counters for each frame
            person_count = 0
            object_count = 0
            business_traveler_count = 0
            family_or_friends_count = 0
            solo_traveler_count = 0
            unattended_objects_count = 0
            # Resize frame if needed
            if frame.shape[0] != TARGET_HEIGHT or frame.shape[1] != TARGET_WIDTH:
                frame = cv2.resize(frame, (TARGET_WIDTH, TARGET_HEIGHT))

            # Detect objects
            results = model(frame)
            print("Detected Objects:", results.xyxy[0])  # Print detected objects

            # Draw bounding boxes, labels, and update counters
            for detection in results.xyxy[0]:
                x1, y1, x2, y2, conf, class_id = map(int, detection[:6])
                class_name = classes[class_id]
                object_id = f"{x1}-{y1}-{x2}-{y2}"  # Unique identifier for object based on bounding box coordinates

                # Determine color based on selected class
                if class_name == selected_class:
                    color = (0, 0, 255)  # Red for selected class
                    object_count=object_count+1
                elif class_name == "person":
                    color = (0, 255, 0)  # Green for person
                    # Update person count and categorize
                    person_count += 1
                    # Get a list of detected objects
                    detected_objects = [detection[4] for detection in results.xyxy[0]]
                    # Categorize person based on detected objects and person count
                    category = categorize_traveler(person_count, detected_objects)
                    # Increment the count based on the category
                    if category == "Family or Friends":
                        family_or_friends_count += 1
                    elif category == "Business Traveler":
                        business_traveler_count += 1
                    else:
                        solo_traveler_count += 1
                else:
                    object_count=object_count+1
                    # Check proximity of luggage class with person
                    if class_name in luggage:
                        person_nearby = False
                        for bbox in results.xyxy[0]:
                            if classes[int(bbox[5])] == "person":
                                person_bbox = [int(coord) for coord in bbox[:4]]
                                person_center = ((person_bbox[0] + person_bbox[2]) // 2, (person_bbox[1] + person_bbox[3]) // 2)
                                object_center = ((x1 + x2) // 2, (y1 + y2) // 2)
                                distance = np.linalg.norm(np.array(person_center) - np.array(object_center))
                                if distance < PERSON_PROXIMITY_THRESHOLD:  # Adjust distance threshold as needed
                                    person_nearby = True
                                    last_near_person_time[class_name] = time.time()  # Update last near person time
                                    break
                        if not person_nearby and time.time() - last_near_person_time.get(class_name,0) > TIME_THRESHOLD:
                            color = (0, 0, 255)  # Red for unattended luggage
                            category = 'Unattended'
                            unattended_objects_count += 1
                            # Set security notification flag
                            security_notification = True
                            # Set priority notification flag
                            priority_notification = True
                            # Send SMS
                            send_sms('security_notification')
                        else:
                            color = (0, 255, 255)  # Yellow
                            category = 'Attended'
                            # Reset security notification flag
                            security_notification = False
                    else:
                        color = (0, 255, 255)
                        object_count=object_count+1

                # Log data
                timestamp = time.strftime("%Y-%m-%d %H:%M:%S")
                log_writer.writerow([timestamp, frame_count, person_count, object_count, unattended_objects_count, business_traveler_count, family_or_friends_count, solo_traveler_count, category, class_name, f"({x1}, {y1})", security_notification, maintenance_notification, priority_notification])

                # Draw bounding box, label, and class color
                cv2.rectangle(frame, (x1, y1), (x2, y2), color, 2)
                cv2.putText(frame, f"{class_name}", (x1, y1 - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

            # Display counters on frame
            font = cv2.FONT_HERSHEY_SIMPLEX
            font_scale = 0.6
            text_thickness = 2

            # People Counters
            cv2.putText(frame, f"People: {person_count}", (10, 20), font, font_scale, (0, 255, 0), text_thickness)

            # Object Counters
            cv2.putText(frame, f"Objects: {object_count}", (frame.shape[1] - 150, 20), font, font_scale, (0, 255, 255), text_thickness)

            # Display the resulting frame
            cv2.imshow('Frame', frame)

            # Break the loop if 'q' is pressed
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break

    # Release resources
    cap.release()
    cv2.destroyAllWindows()


def start_processing():
    selected_class = class_combobox.get()
    # Define camera parameters
    # Define camera parameters
    camera_ip = ''  # Replace with the actual IP address of the Bosch camera
    #camera_ip = ''  # Replace with the actual IP address of the Bosch camera
    username = ''  # Replace with the actual username for camera authentication
    password = ''  # Replace with the actual password for camera authentication

    # Define video stream URL
    stream_url = f'rtsp://{username}:{password}@{camera_ip}/video'

    # Open video stream
    cap = cv2.VideoCapture(stream_url)
#     cap = cv2.VideoCapture(0)

    # Check if the camera stream is opened successfully
    if not cap.isOpened():
        print("Error: Could not open camera stream.")
        exit()

    log_file = "object_log.csv"  # Name of the log file
    model = torch.hub.load('ultralytics/yolov5', 'yolov5m', pretrained=True)
    process_frames(cap, model, selected_class, log_file)


if __name__ == "__main__":
    root = tk.Tk()
    root.title("Object Detection")
    root.geometry("300x150")

    # Dropdown list to select the class
    luggage = ["handbag", "suitcase", "backpack"]
    classes = ["wallet", "laptop", "cell phone", "remote", "handbag", "suitcase", "backpack"]
    class_combobox = ttk.Combobox(root, values=classes)
    class_combobox.set("Select Class")
    class_combobox.pack()

    button = tk.Button(root, text="Start Processing", command=start_processing)
    button.pack()

    root.mainloop()

Using cache found in C:\Users\LENOVO/.cache\torch\hub\ultralytics_yolov5_master
YOLOv5  2024-4-8 Python-3.9.13 torch-2.2.2 CPU

Fusing layers... 
YOLOv5m summary: 290 layers, 21172173 parameters, 0 gradients, 48.9 GFLOPs
Adding AutoShape... 


Detected Objects: tensor([[5.03715e+02, 3.58262e+02, 5.90477e+02, 4.79377e+02, 9.24482e-01, 0.00000e+00],
        [5.07354e+01, 2.16633e+02, 1.11004e+02, 4.28751e+02, 8.94172e-01, 0.00000e+00],
        [9.80486e+00, 1.04000e+02, 4.78011e+01, 2.54041e+02, 8.22950e-01, 0.00000e+00],
        [0.00000e+00, 2.25488e+02, 3.50362e+01, 4.41825e+02, 8.18140e-01, 0.00000e+00],
        [8.58527e+01, 1.58812e+02, 1.42429e+02, 3.68579e+02, 7.88898e-01, 0.00000e+00],
        [2.23163e+02, 1.68684e+02, 2.63203e+02, 3.34245e+02, 7.46590e-01, 0.00000e+00],
        [3.67408e+02, 2.80826e+02, 4.30987e+02, 3.52340e+02, 7.43522e-01, 0.00000e+00],
        [1.16989e+02, 4.19133e+02, 1.61779e+02, 4.80000e+02, 7.15187e-01, 0.00000e+00],
        [1.43359e+02, 3.76944e+02, 2.16585e+02, 4.79109e+02, 6.85410e-01, 0.00000e+00],
        [1.86947e+01, 4.19192e+02, 7.20932e+01, 4.80000e+02, 6.80221e-01, 0.00000e+00],
        [2.68136e+02, 1.95421e+02, 3.12149e+02, 3.28762e+02, 6.77832e-01, 0.00000e+00],
        [2.136

Detected Objects: tensor([[5.07929e+01, 2.17466e+02, 1.10310e+02, 4.28481e+02, 8.98230e-01, 0.00000e+00],
        [5.46446e+02, 3.43078e+02, 6.21091e+02, 4.80000e+02, 8.68148e-01, 0.00000e+00],
        [5.98133e+00, 1.05566e+02, 4.78893e+01, 2.53095e+02, 8.49722e-01, 0.00000e+00],
        [2.70373e+02, 1.96331e+02, 3.17051e+02, 3.26545e+02, 8.25811e-01, 0.00000e+00],
        [0.00000e+00, 2.26532e+02, 3.52206e+01, 4.40870e+02, 8.21654e-01, 0.00000e+00],
        [2.23916e+02, 1.66930e+02, 2.66569e+02, 3.35855e+02, 7.71550e-01, 0.00000e+00],
        [1.17271e+02, 4.18868e+02, 1.62172e+02, 4.80000e+02, 7.34149e-01, 0.00000e+00],
        [3.67047e+02, 2.77952e+02, 4.30460e+02, 3.53292e+02, 6.98606e-01, 0.00000e+00],
        [1.44167e+02, 3.76911e+02, 2.16638e+02, 4.79112e+02, 6.70023e-01, 0.00000e+00],
        [8.59297e+01, 1.65645e+02, 1.44031e+02, 3.72683e+02, 6.59455e-01, 0.00000e+00],
        [2.13494e+02, 3.26153e+02, 2.75525e+02, 4.80000e+02, 6.48157e-01, 0.00000e+00],
        [1.891

Detected Objects: tensor([[3.68359e+01, 2.23107e+02, 1.12539e+02, 4.17999e+02, 8.82358e-01, 0.00000e+00],
        [5.69118e+02, 3.55482e+02, 6.40000e+02, 4.80000e+02, 8.57319e-01, 0.00000e+00],
        [9.06765e+00, 1.05529e+02, 4.79615e+01, 2.54433e+02, 8.11572e-01, 0.00000e+00],
        [8.62130e+01, 1.75531e+02, 1.58475e+02, 3.73861e+02, 8.05765e-01, 0.00000e+00],
        [0.00000e+00, 2.27338e+02, 3.64659e+01, 4.46107e+02, 7.90591e-01, 0.00000e+00],
        [2.72392e+02, 2.02991e+02, 3.18978e+02, 3.25091e+02, 7.73702e-01, 0.00000e+00],
        [2.19651e+02, 1.68602e+02, 2.63783e+02, 3.30776e+02, 7.62178e-01, 0.00000e+00],
        [1.16766e+02, 4.19019e+02, 1.61867e+02, 4.79964e+02, 7.20928e-01, 0.00000e+00],
        [2.13848e+02, 3.22031e+02, 2.74289e+02, 4.80000e+02, 6.78783e-01, 0.00000e+00],
        [1.44475e+02, 3.76806e+02, 2.16307e+02, 4.78955e+02, 6.72690e-01, 0.00000e+00],
        [3.68336e+02, 2.78930e+02, 4.29903e+02, 3.54096e+02, 6.61017e-01, 0.00000e+00],
        [2.689

Detected Objects: tensor([[3.70760e+01, 2.22230e+02, 1.12979e+02, 4.18069e+02, 9.03996e-01, 0.00000e+00],
        [1.15251e+02, 1.71798e+02, 1.70501e+02, 3.89928e+02, 8.65223e-01, 0.00000e+00],
        [9.60481e+00, 1.04953e+02, 4.78042e+01, 2.53301e+02, 8.45172e-01, 0.00000e+00],
        [0.00000e+00, 2.26967e+02, 3.65286e+01, 4.45340e+02, 8.28217e-01, 0.00000e+00],
        [2.72324e+02, 2.05010e+02, 3.18276e+02, 3.25592e+02, 8.11742e-01, 0.00000e+00],
        [2.10251e+02, 1.73941e+02, 2.60152e+02, 3.36386e+02, 7.79563e-01, 0.00000e+00],
        [2.02545e+02, 3.28649e+02, 2.72276e+02, 4.80000e+02, 7.40175e-01, 0.00000e+00],
        [1.16680e+02, 4.18610e+02, 1.61375e+02, 4.79986e+02, 7.34639e-01, 0.00000e+00],
        [3.67964e+02, 2.77341e+02, 4.29587e+02, 3.52710e+02, 7.03134e-01, 0.00000e+00],
        [1.44307e+02, 3.77410e+02, 2.16298e+02, 4.79009e+02, 6.92458e-01, 0.00000e+00],
        [2.02352e+01, 4.18003e+02, 7.15760e+01, 4.79900e+02, 6.49524e-01, 0.00000e+00],
        [1.594

...

Detected Objects: tensor([[3.74173e+02, 2.77192e+02, 4.26829e+02, 3.55660e+02, 8.20164e-01, 0.00000e+00],
        [5.70071e+02, 3.82862e+02, 6.37787e+02, 4.80000e+02, 6.62121e-01, 0.00000e+00],
        [2.68068e+01, 1.29650e+02, 6.73165e+01, 2.92381e+02, 6.34963e-01, 0.00000e+00],
        [6.06642e-02, 1.58151e+02, 3.02114e+01, 3.08380e+02, 6.29194e-01, 0.00000e+00],
        [2.39676e+02, 1.95873e+02, 2.87879e+02, 3.12207e+02, 6.10821e-01, 0.00000e+00],
        [2.87593e+02, 1.93656e+02, 3.35137e+02, 3.25173e+02, 6.05171e-01, 0.00000e+00],
        [1.27018e+02, 3.73820e+02, 2.12926e+02, 4.78527e+02, 6.01316e-01, 0.00000e+00],
        [3.10990e+02, 2.88651e+02, 3.85225e+02, 3.77810e+02, 6.00864e-01, 0.00000e+00],
        [2.13587e+02, 3.36188e+02, 2.79246e+02, 4.78986e+02, 6.00171e-01, 0.00000e+00],
        [2.46689e+02, 3.01153e+02, 3.25666e+02, 4.80000e+02, 4.99709e-01, 0.00000e+00],
        [3.63911e+02, 1.56600e+02, 3.96880e+02, 2.15030e+02, 4.92494e-01, 2.40000e+01],
        [5.027

Detected Objects: tensor([[5.66074e+02, 3.86649e+02, 6.24861e+02, 4.79742e+02, 8.37220e-01, 0.00000e+00],
        [3.74051e+02, 2.76580e+02, 4.26925e+02, 3.55595e+02, 8.15814e-01, 0.00000e+00],
        [2.88035e+02, 1.92507e+02, 3.32969e+02, 3.23699e+02, 7.06222e-01, 0.00000e+00],
        [2.46983e+01, 1.29277e+02, 6.70954e+01, 2.92296e+02, 7.00382e-01, 0.00000e+00],
        [1.27876e+02, 3.74161e+02, 2.17712e+02, 4.78866e+02, 6.91378e-01, 0.00000e+00],
        [2.14346e+02, 3.35970e+02, 2.79557e+02, 4.79097e+02, 6.27858e-01, 0.00000e+00],
        [3.11420e+02, 2.87836e+02, 3.83898e+02, 3.77639e+02, 6.03187e-01, 0.00000e+00],
        [3.48847e+02, 1.52856e+02, 3.96921e+02, 2.10594e+02, 6.01092e-01, 2.40000e+01],
        [2.39032e+02, 1.96231e+02, 2.88334e+02, 3.12334e+02, 5.49749e-01, 0.00000e+00],
        [2.00389e+02, 3.59147e+02, 2.28077e+02, 4.65123e+02, 4.98107e-01, 5.60000e+01],
        [3.11245e+02, 1.49615e+02, 3.65839e+02, 3.17967e+02, 4.80114e-01, 0.00000e+00],
        [7.990

Detected Objects: tensor([[5.19339e+02, 8.18499e+01, 5.46487e+02, 2.15132e+02, 8.54598e-01, 0.00000e+00],
        [3.14083e-02, 1.32072e+02, 3.62591e+01, 3.05844e+02, 8.01865e-01, 0.00000e+00],
        [3.13388e+02, 2.90228e+02, 3.87881e+02, 3.76523e+02, 7.83121e-01, 0.00000e+00],
        [5.60496e+02, 3.95880e+02, 6.10406e+02, 4.79521e+02, 7.70682e-01, 0.00000e+00],
        [3.73949e+02, 2.84574e+02, 4.36490e+02, 3.56910e+02, 7.67733e-01, 0.00000e+00],
        [3.60031e+01, 1.27796e+02, 8.28186e+01, 2.85991e+02, 7.51511e-01, 0.00000e+00],
        [2.72300e+02, 3.03524e+02, 3.40633e+02, 3.89524e+02, 7.32000e-01, 0.00000e+00],
        [1.29509e+02, 3.73823e+02, 2.21456e+02, 4.79333e+02, 7.09862e-01, 0.00000e+00],
        [2.87879e+02, 1.91066e+02, 3.34997e+02, 3.22808e+02, 7.00085e-01, 0.00000e+00],
        [2.26034e+01, 8.39297e+01, 5.00386e+01, 1.73488e+02, 6.88295e-01, 0.00000e+00],
        [2.14037e+02, 3.37055e+02, 2.79287e+02, 4.78615e+02, 6.58760e-01, 0.00000e+00],
        [2.372

Detected Objects: tensor([[5.18530e+02, 8.11248e+01, 5.43650e+02, 2.15462e+02, 8.11498e-01, 0.00000e+00],
        [2.04537e+02, 3.18940e+02, 2.81698e+02, 4.80000e+02, 7.89062e-01, 0.00000e+00],
        [2.70160e+02, 3.03516e+02, 3.45150e+02, 3.91106e+02, 7.88597e-01, 0.00000e+00],
        [5.62223e+02, 3.93955e+02, 6.10621e+02, 4.79956e+02, 7.84147e-01, 0.00000e+00],
        [1.93148e-01, 1.37133e+02, 3.45336e+01, 3.05814e+02, 7.68679e-01, 0.00000e+00],
        [3.73965e+02, 2.84264e+02, 4.37902e+02, 3.57725e+02, 7.54380e-01, 0.00000e+00],
        [2.88680e+02, 1.94365e+02, 3.34790e+02, 3.23854e+02, 7.43593e-01, 0.00000e+00],
        [3.62001e+01, 1.29115e+02, 8.46318e+01, 2.86573e+02, 7.22444e-01, 0.00000e+00],
        [3.15430e+02, 2.91436e+02, 3.87634e+02, 3.73966e+02, 7.21861e-01, 0.00000e+00],
        [2.36906e+02, 1.95975e+02, 2.88884e+02, 3.37376e+02, 7.04836e-01, 0.00000e+00],
        [1.32097e+02, 3.74806e+02, 2.20429e+02, 4.78871e+02, 6.83551e-01, 0.00000e+00],
        [3.277

Detected Objects: tensor([[5.18336e+02, 8.08278e+01, 5.43425e+02, 2.15264e+02, 8.43018e-01, 0.00000e+00],
        [2.06221e+02, 3.22662e+02, 2.81516e+02, 4.80000e+02, 7.91847e-01, 0.00000e+00],
        [3.59433e+01, 1.27936e+02, 8.21099e+01, 2.85887e+02, 7.65740e-01, 0.00000e+00],
        [3.82773e-01, 1.61575e+02, 2.86513e+01, 3.10358e+02, 7.60656e-01, 0.00000e+00],
        [1.31668e+02, 3.74875e+02, 2.08854e+02, 4.79307e+02, 7.28508e-01, 0.00000e+00],
        [2.88139e+02, 1.92655e+02, 3.34980e+02, 3.24374e+02, 7.27905e-01, 0.00000e+00],
        [3.12855e+02, 2.93211e+02, 3.81213e+02, 3.71194e+02, 7.23813e-01, 0.00000e+00],
        [2.38141e+02, 1.96018e+02, 2.87665e+02, 3.36038e+02, 6.91234e-01, 0.00000e+00],
        [2.68801e+02, 3.03770e+02, 3.33325e+02, 3.96067e+02, 6.54239e-01, 0.00000e+00],
        [2.35363e+01, 8.43349e+01, 4.94834e+01, 1.44582e+02, 6.43325e-01, 0.00000e+00],
        [3.74757e+02, 2.78358e+02, 4.35502e+02, 3.58110e+02, 6.34113e-01, 0.00000e+00],
        [2.582